# MILO - Cell type composition analysis

Publication [Dann et al, 2021](https://www.nature.com/articles/s41587-021-01033-z)  
Marioni Lab [GitHub](https://github.com/MarioniLab/miloR)  

Theis lab python adaption with [pertpy](https://github.com/theislab/pertpy)

In [ ]:
options(warn=-1)

In [ ]:
library_load <- suppressMessages(
    
    suppressWarnings(
        
        list(
        
            # Seurat 
            library(Seurat), 

            # miloR
            library(miloR), 
            library(ggbeeswarm), 
            
            # SingleCellExperiment
            library(SingleCellExperiment), 

            # Data 
            library(tidyverse), 
            library(data.table), 

            # Plotting 
            library(patchwork), 
            library(cowplot), 
            
            # Parallel 
            library(BiocParallel), 

            # Pyhton compatibility
            library(reticulate)

        )
    )
)

In [ ]:
# Configure reticulate 
use_condaenv(condaenv="p.3.10.16-FD20200109SPLENO", conda="/nobackup/peer/fdeckert/miniconda3/bin/conda", required=NULL)
py_config()

In [ ]:
random_seed <- 42
set.seed(random_seed)

In [ ]:
# Set working directory to project root
setwd("/research/peer/fdeckert/FD20200109SPLENO")

In [ ]:
# Plotting Theme
source("plotting_global.R")
ggplot2::theme_set(theme_global_set(size_select=1)) # From project global source()

# Import Milo object

In [ ]:
milo <- readRDS(paste0("data/scRNAseq/milo/milo_k30.rds"))

In [ ]:
results <- readRDS("data/scRNAseq/milo/resl.rds")

In [ ]:
result <- results[[3]][["res_da"]][[1]]

# Plot results

In [ ]:
alpha <- 0.1
group.by <- "celltype_low"

In [ ]:
# Beeswarm 
data <- suppressMessages(plotDAbeeswarm(result, group.by=group.by, alpha=alpha))

In [ ]:
data <- data$data %>%
    mutate(is_signif = ifelse(SpatialFDR < alpha, 1, 0)) %>%
    mutate(logFC_color = ifelse(is_signif==1, logFC, NA)) %>%
    arrange(group_by) %>%
    mutate(Nhood=factor(Nhood, levels=unique(Nhood))) %>%
    mutate(pos_x=pos_x, pos_y=pos_y) 

In [ ]:
y_breaks <- c(-floor(max(abs(data$logFC))), -floor(max(abs(data$logFC)))/2, 0, floor(max(abs(data$logFC)))/2, floor(max(abs(data$logFC))))
n_groups <- length(unique(data$group_by))

In [ ]:
options(repr.plot.width=4, repr.plot.height=10)

dabp <- ggplot(data, aes(pos_x, pos_y, color=logFC_color)) +
    scale_color_gradient2(low=color$infection["NaCl"], high=color$infection["CpG"]) + 
    guides(color="none") +
    xlab("") + ylab("Log Fold Change") + ggtitle("Differential abundance") + 
    scale_x_discrete(labels=setNames(levels(data$group_by), seq(1,n_groups))) + 
    scale_y_continuous(breaks=y_breaks, labels=y_breaks) +
    geom_hline(yintercept=0, size=1) + geom_point(size=3) + stat_summary(aes(x=group_by, y=logFC), fun.y=median, geom="point", size=2, color="black") +
    coord_flip() +
    theme(axis.text.y=element_text(vjust=0.5, hjust=1)) + 
    theme_global_set(1)

dabp

# Session info

In [ ]:
sessionInfo()